In [25]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
%matplotlib inline

In [1]:
# !apt-get update
# !sudo apt-get install openjdk-8-jre
# !apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
# !tar xf spark-2.3.1-bin-hadoop2.7.tgz
# !sudo pip install findspark

import os
os.environ["JAVA_HOME"] = "/Users/SDMAN/Downloads/jdk-14.0.1.jdk/Contents/Home"
os.environ["SPARK_HOME"] = "/Users/SDMAN/spark-2.4.5-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark

Anime_Recommender.ipynb         Pyspark_Anime_Recommender.ipynb
Pipfile                         myAnimelist-No_Hentai 2.csv
Pipfile.lock                    rating.csv


In [2]:
from pyspark import SparkContext
sc = SparkContext()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by getOrCreate at <ipython-input-1-f67d597937cb>:19 

In [5]:
anime_data  = spark.read.csv("myAnimelist-No_Hentai 2.csv",header = True,inferSchema=True)
rating_data = spark.read.csv("/Users/SDMAN/Documents/Flatiron/Projects/Anime Recommender System/rating.csv",header = True,inferSchema=True)

In [6]:
anime_data.show()
rating_data.show()

+-------+--------------------+-----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+----------+-------+---------+
|animeID|                name| type|     source|episodes|duration|               genre|              rating|score|scored_by|rank|popularity|members|favorites|
+-------+--------------------+-----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+----------+-------+---------+
|      1|        Cowboy Bebop|   TV|   Original|      26| 0:24:00|['Action', 'Adven...|R - 17+ (violence...| 8.81| 405664.0|  26|        39| 795733|    43460|
|      5|Cowboy Bebop: Ten...|Movie|   Original|       1| 1:55:00|['Action', 'Drama...|R - 17+ (violence...| 8.41| 120243.0| 164|       449| 197791|      776|
|      6|              Trigun|   TV|      Manga|      26| 0:24:00|['Action', 'Sci-F...|PG-13 - Teens 13 ...|  8.3| 212537.0| 255|       146| 408548|    10432|
|      7|  Witch Hunter Robin|   TV|   Origina

In [7]:
from pyspark.sql.functions import *

In [8]:
#Replacing values in column rating from -1 to null values to not mess with mean calculations 
rating_data2 = rating_data.withColumn(
    'rating',
    when(
        col('rating').isin(-1),None
    ).otherwise(col('rating'))
)
#Renaming column rating with user_rating to not confuse with anime dataframe's rating column
rating_data2 = rating_data2.withColumnRenamed("rating", "user_rating")
rating_data2.printSchema()
rating_data2.show()


root
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- user_rating: integer (nullable = true)

+-------+--------+-----------+
|user_id|anime_id|user_rating|
+-------+--------+-----------+
|      1|      20|       null|
|      1|      24|       null|
|      1|      79|       null|
|      1|     226|       null|
|      1|     241|       null|
|      1|     355|       null|
|      1|     356|       null|
|      1|     442|       null|
|      1|     487|       null|
|      1|     846|       null|
|      1|     936|       null|
|      1|    1546|       null|
|      1|    1692|       null|
|      1|    1836|       null|
|      1|    2001|       null|
|      1|    2025|       null|
|      1|    2144|       null|
|      1|    2787|       null|
|      1|    2993|       null|
|      1|    3455|       null|
+-------+--------+-----------+
only showing top 20 rows



In [18]:
# only will work with TV shows and no other type
anime_shows = anime_data[anime_data['type'] == 'TV']
anime_shows.show()

+-------+--------------------+----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+----------+-------+---------+
|animeID|                name|type|     source|episodes|duration|               genre|              rating|score|scored_by|rank|popularity|members|favorites|
+-------+--------------------+----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+----------+-------+---------+
|      1|        Cowboy Bebop|  TV|   Original|      26| 0:24:00|['Action', 'Adven...|R - 17+ (violence...| 8.81| 405664.0|  26|        39| 795733|    43460|
|      6|              Trigun|  TV|      Manga|      26| 0:24:00|['Action', 'Sci-F...|PG-13 - Teens 13 ...|  8.3| 212537.0| 255|       146| 408548|    10432|
|      7|  Witch Hunter Robin|  TV|   Original|      26| 0:25:00|['Action', 'Magic...|PG-13 - Teens 13 ...| 7.33|  32837.0|2371|      1171|  79397|      537|
|      8|      Bouken Ou Beet|  TV|      Manga|     

In [19]:
# Renaming column in dataframe in pyspark
df = anime_shows.withColumnRenamed("animeID", "anime_id")
df.printSchema()
df.show()

root
 |-- anime_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- score: string (nullable = true)
 |-- scored_by: double (nullable = true)
 |-- rank: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- members: integer (nullable = true)
 |-- favorites: integer (nullable = true)

+--------+--------------------+----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+----------+-------+---------+
|anime_id|                name|type|     source|episodes|duration|               genre|              rating|score|scored_by|rank|popularity|members|favorites|
+--------+--------------------+----+-----------+--------+--------+--------------------+--------------------+-----+---------+----+-

In [25]:
#Merge Anime TV shows DataFrame with rating dataFrame
anime_rating=anime_shows.join(rating_data2, anime_shows['animeID']==rating_data2['anime_id'])
anime_rating.printSchema()
anime_rating.collect()

root
 |-- animeID: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- type: string (nullable = true)
 |-- source: string (nullable = true)
 |-- episodes: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- genre: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- score: string (nullable = true)
 |-- scored_by: double (nullable = true)
 |-- rank: integer (nullable = true)
 |-- popularity: integer (nullable = true)
 |-- members: integer (nullable = true)
 |-- favorites: integer (nullable = true)
 |-- user_id: integer (nullable = true)
 |-- anime_id: integer (nullable = true)
 |-- user_rating: integer (nullable = true)



Py4JJavaError: An error occurred while calling o998.collectToPython.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.exec

In [ ]:
anime_rating.count()

In [21]:
piv_anime_rating = anime_rating.groupBy('user_id').pivot('name').mean('user_rating')
piv_anime_rating.show()

Py4JJavaError: An error occurred while calling o613.pivot.
: org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:146)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:387)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:144)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeBroadcast$1.apply(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.SparkPlan$$anonfun$executeQuery$1.apply(SparkPlan.scala:155)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:152)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:140)
	at org.apache.spark.sql.execution.join

In [20]:
from pyspark.sql.functions import pandas_udf, PandasUDFType


def normalize(pdf):
    v = pdf.v
    return pdf.assign(v=(v - v.mean()) / v.std())


piv_anime_rating.groupby("user_id").apply(normalize).show()

NameError: name 'piv_anime_rating' is not defined